<a href="https://colab.research.google.com/github/akanison12/Sudo-code-2024/blob/main/240114_ML_for_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Library

In [ ]:
!pip install kaggle
!pip install underthesea
! pip install vietnam-number
!kaggle datasets download -d tuannguyenvananh/vietnamese-text-classification-dataset
!unzip vietnamese-text-classification-dataset.zip



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
Dataset URL: https://www.kaggle.com/datasets/tuannguyenvananh/vietnamese-text-classification-dataset
License(s): unknown
100% 173k/173k [00:00<00:00, 482kB/s]
100% 173k/173k [00:00<00:00, 481kB/s]
Archive:  vietnamese-text-classification-dataset.zip
  inflating: train.csv               


In [ ]:
! git clone https://github.com/stopwords/vietnamese-stopwords

Cloning into 'vietnamese-stopwords'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 95 (delta 3), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (95/95), 40.25 KiB | 597.00 KiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import underthesea
from vietnam_number import n2w
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from underthesea import word_tokenize
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier


#Read file

In [ ]:
file_path='/content/train.csv'

text_data = pd.read_csv(file_path, header=None, names=['label', 'content'])
text_data


,label,content
0,0,máy dùng hay bị đơ máy
1,0,chỉ có dây cáp nguồn không có adapter sao sử d...
2,0,Chất lượng quá kém Mới dùng được 2 ngày loa ba...
3,0,Usb tôi vừa mới nhận usb này Rất bực bội vì cá...
4,2,Tuyệt vời. Hàng FPT cửa hàng
...,...,...
3035,0,không có bộ thu đi kèm Vừa nhận hàng xong tức ...
3036,0,Cũng bình thường Không mạnh như kỹ vọng Chắc t...
3037,2,tốt tốt đang tiền khi bỏ ra để mua giao hàng n...
3038,0,giao hàng lâu rửa tay một tí đã cháy màn hình ...


#PreProcessing

In [ ]:
class Preprocessing:

    def processing(self, text):
        """Process input text."""

        # Convert to lowercase and remove excess whitespace
        text = text.strip().lower()

        # Replace numbers with the suffix 'k' to 'ngàn' (thousand)
        text = re.sub(r'(\d+)k', r'\1 ngàn', text)

        # Replace % with 'phần trăm' (percent)
        text = re.sub(r'(\d+)%', r'\1 phần trăm', text)

        # Replace 'm' with 'mét' (meters)
        text = re.sub(r'(\d+)m(?![a-zA-Z])', r'\1 mét', text)

        # Replace 's' with 'giây' (seconds)
        text = re.sub(r'(\d+)s(?![a-zA-Z])', r'\1 giây', text)

        # Replace ' with 'phút' (minutes)
        text = re.sub(r'(\d+)\'(?![a-zA-Z])', r'\1 phút', text)

        # Replace 'h' with 'giờ' (hours)
        text = re.sub(r'(\d+)h(?![a-zA-Z])', r'\1 giờ', text)

        # Convert numbers to words if they exist
        text = re.sub(r'\d+', lambda x: n2w(x.group()), text)

        # Replace special characters with an empty string
        text = re.sub(r'[^\w\s]', '', text)

        # Normalize Vietnamese text
        text = underthesea.text_normalize(text)

        # Tokenize the text
        words = underthesea.word_tokenize(text, format="text")

        return words











In [ ]:

preprocessor = Preprocessing()

text_data['preprocessed_content'] = text_data['content'].apply(preprocessor.processing)
text_data

,label,content,preprocessed_content
0,0,máy dùng hay bị đơ máy,máy dùng hay bị đơ máy
1,0,chỉ có dây cáp nguồn không có adapter sao sử d...,chỉ có dây_cáp nguồn không có adapter sao sử_d...
2,0,Chất lượng quá kém Mới dùng được 2 ngày loa ba...,chất_lượng quá kém mới dùng được hai ngày loa ...
3,0,Usb tôi vừa mới nhận usb này Rất bực bội vì cá...,usb tôi vừa_mới nhận usb này rất bực_bội vì cá...
4,2,Tuyệt vời. Hàng FPT cửa hàng,tuyệt_vời hàng fpt cửa_hàng
...,...,...,...
3035,0,không có bộ thu đi kèm Vừa nhận hàng xong tức ...,không có bộ thu đi kèm vừa nhận hàng xong tức_...
3036,0,Cũng bình thường Không mạnh như kỹ vọng Chắc t...,cũng bình_thường không mạnh như kỹ_vọng chắc t...
3037,2,tốt tốt đang tiền khi bỏ ra để mua giao hàng n...,tốt tốt đang tiền khi bỏ ra để mua giao hàng n...
3038,0,giao hàng lâu rửa tay một tí đã cháy màn hình ...,giao hàng lâu rửa_tay một_tí đã cháy màn_hình ...


#Check Data Balanced

In [ ]:
text_data['label'].value_counts()


,count
label,
0,1105
2,1048
1,887


Comment:Data is Balanced

#Train Model

In [ ]:

X = text_data['preprocessed_content']
y = text_data['label']


#StratifiedKFold
Stratified K-Fold cross-validator.

Provides train/test indices to split data in train/test sets.

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [ ]:

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#Tf-IDf with  MultimonialNaive Bayes
The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification).

In [ ]:



accuracy_list_nb = []

# Cross-validation For Naive Bayes
print("Cross-validation Naive Bayes:")
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create pipeline  Naive Bayes
    nb_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('nb',MultinomialNB())
    ])

    #Train
    nb_pipeline.fit(X_train, y_train)
    y_pred_nb = nb_pipeline.predict(X_test)


    accuracy = accuracy_score(y_test, y_pred_nb)
    accuracy_list_nb.append(accuracy)
    print("Accuracy:", accuracy)

# Tính Avaerage Accuracy
average_accuracy_nb = sum(accuracy_list_nb) / len(accuracy_list_nb)
print("\nAverage Accuracy for Naive Bayes:", average_accuracy_nb)



Cross-validation Naive Bayes:
Accuracy: 0.7483552631578947
Accuracy: 0.7450657894736842
Accuracy: 0.7713815789473685
Accuracy: 0.7615131578947368
Accuracy: 0.7631578947368421

Average Accuracy for Naive Bayes: 0.7578947368421052


#Tf-IDF with One with Rest SVM
 this strategy consists in fitting one classifier per class. For each classifier, the class is fitted against all the other classes. In addition to its computational efficiency (only n_classes classifiers are needed), one advantage of this approach is its interpretability.


In [ ]:
svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', OneVsRestClassifier(SVC()))
])

accuracy_list_svm = []

print("\nCross-validation SVM:")
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    svm_pipeline.fit(X_train, y_train)
    y_pred_svm = svm_pipeline.predict(X_test)


    accuracy = accuracy_score(y_test, y_pred_svm)
    accuracy_list_svm.append(accuracy)
    print("Accuracy:", accuracy)

average_accuracy_svm = sum(accuracy_list_svm) / len(accuracy_list_svm)
print("\nAverage Accuracy for SVM:", average_accuracy_svm)





Cross-validation SVM:
Accuracy: 0.8125
Accuracy: 0.7993421052631579
Accuracy: 0.819078947368421
Accuracy: 0.8125
Accuracy: 0.8026315789473685

Average Accuracy for SVM: 0.8092105263157894


#Word2vec

In [ ]:
# Tokenize the text
tokenized_text = [word_tokenize(text, format="text").split() for text in X]

# Train the Word2Vec model (if the model does not exist yet)
w2v_model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)

# Build the vocabulary
w2v_model.build_vocab(tokenized_text)

# Train the model
w2v_model.train(tokenized_text, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)

# Step 2: Convert text into vectors by averaging the Word2Vec vectors
X_vectors = np.array([
    np.mean([w2v_model.wv[word] for word in text.split() if word in w2v_model.wv], axis=0)
    if len(text.split()) > 0 else np.zeros(w2v_model.vector_size)
    for text in X
])

# Step 3: Use Stratified K-Fold for cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)






In [ ]:
tokenized_text

[['máy', 'dùng', 'hay', 'bị', 'đơ', 'máy'],
 ['chỉ',
  'có',
  'dây_cáp',
  'nguồn',
  'không',
  'có',
  'adapter',
  'sao',
  'sử_dụng',
  'chỉ',
  'có',
  'dây_cáp',
  'nguồn',
  'không',
  'có',
  'adapter',
  'sao',
  'sử_dụng',
  'hàng',
  'tiki_training',
  'gì',
  'kì_cục',
  'vậy'],
 ['chất_lượng',
  'quá',
  'kém',
  'mới',
  'dùng',
  'được',
  'hai',
  'ngày',
  'loa',
  'bass',
  'có',
  'âm_thanh',
  'rè_rè',
  'dù',
  'không',
  'mở',
  'gì',
  'không',
  'tin_tưởng',
  'được',
  'chất_lượng'],
 ['usb',
  'tôi',
  'vừa_mới',
  'nhận',
  'usb',
  'này',
  'rất',
  'bực_bội',
  'vì',
  'cái',
  'nắp',
  'usb',
  'này',
  'rất',
  'cứng_không_thể',
  'mở',
  'ra',
  'được',
  'và',
  'cái',
  'hộp',
  'đựng',
  'usb',
  'này',
  'dường_như',
  'bị',
  'ai',
  'đó',
  'cắt',
  'ra',
  'trước',
  'đó'],
 ['tuyệt_vời', 'hàng', 'fpt', 'cửa_hàng'],
 ['chưa',
  'đo',
  'đúng',
  'bước',
  'chân',
  'rõ',
  'nhiều',
  'khi',
  'gián_đoạn',
  'chưa',
  'hài_lòng',
  'hàng_không',
 

#Word2vec with Gaussian Naive Bayes

In [ ]:
# Inital GaussianNB
nb_model = GaussianNB()

# Cross-validation for Naive Bayes
print("Cross-validation Naive Bayes:")
for train_index, test_index in kf.split(X_vectors, y):
    X_train, X_test = X_vectors[train_index], X_vectors[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train
    nb_model.fit(X_train, y_train)
    y_pred_nb = nb_model.predict(X_test)

    #Score
    accuracy = accuracy_score(y_test, y_pred_nb)
    print("Accuracy:", accuracy)

Cross-validation Naive Bayes:
Accuracy: 0.4819078947368421
Accuracy: 0.4605263157894737
Accuracy: 0.5296052631578947
Accuracy: 0.4457236842105263
Accuracy: 0.5115131578947368


#Word2vec with SVM

In [ ]:
#  SVM inital
svm_model = OneVsRestClassifier(SVC())


# Cross-validation with SVM
print("\nCross-validation SVM:")
for train_index, test_index in kf.split(X_vectors, y):
    X_train, X_test = X_vectors[train_index], X_vectors[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train SVM
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)

    # Score
    accuracy = accuracy_score(y_test, y_pred_svm)
    print("Accuracy:", accuracy)


Cross-validation SVM:
Accuracy: 0.5773026315789473
Accuracy: 0.5805921052631579
Accuracy: 0.5756578947368421
Accuracy: 0.555921052631579
Accuracy: 0.569078947368421
